In [137]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [23]:
## FEATURE SELECTION IMPORTS

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

#import statsmodels.api as sm

from pandas.core import datetools
%matplotlib inline

In [12]:
missiles = pd.read_csv("data_missile_coordinate_distanceGaza.csv")

missiles = pd.read_csv("missileStrikeCounts.csv").drop(columns=["Unnamed: 0"])

In [13]:
census = pd.read_csv("census_with_gaza-Copy1.csv")

In [19]:
data = pd.merge(census, missiles, how="left", right_on="place", left_on="LocNameHeb")
#sum(data["missile"].isnull())
#sum(data.missile)
data["count"] = data["count"].fillna(0)
sum(data["count"])

#pd.DataFrame(census.LocNameHeb.unique()).to_csv("census_names.csv")

381632.0

In [20]:
new_data = pd.get_dummies(data)

In [21]:
# Cleaning 
#Model
new_data.columns

Index(['Unnamed: 0', 'latitude', 'longitude', 'Unnamed: 0.1',
       'Acadm1Cert_pcnt', 'Acadm2Cert_pcnt', 'AcadmCert_pcnt', 'Bicycle_pcnt',
       'Bus_pcnt', 'CarDrvr_pcnt',
       ...
       'place_y_שקף', 'place_y_שתולים', 'place_y_תימורים',
       'place_y_תלמי אלעזר', 'place_y_תלמי יחיאל', 'place_y_תלמים',
       'place_y_תנובות', 'place_y_תקומה', 'place_y_תקוע', 'place_y_תרום'],
      dtype='object', length=2825)

In [84]:
missing_data = [i for i, x in enumerate((X.isna().sum() > 100)) if x]
new_data.columns[missing_data]

uint_data = [i for i, x in enumerate((X.dtypes == "uint8")) if x]

In [120]:
X = new_data.drop(columns="count")
y = new_data["count"]
X = X.drop(columns=X.columns[missing_data])
uint_data = [i for i, x in enumerate((X.dtypes == "uint8")) if x]
X = X.drop(columns=X.columns[uint_data])
#get rid of infinitys
X.replace([np.inf, -np.inf], np.nan).dropna(subset=X.columns, how="all")

# Fix NA's, fill data with mean
for column in X.columns:
    X[column] = X[column].fillna(X[column].mean())
    
# split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

6630

In [123]:
### FEATURE SELECTION

# dataset_x should be a pandas dataframe

## accepts dataset inputs as numpy arrays
def make_bootstrap_sample(dataset_X, dataset_y, size = None):
    """Returns a bootstrap sample of the given data and size
    
    Args:
        dataset_X: the Pandas dataframe of feature variables and values
        dataset_y: the column vector of response values
        size: the size of the Bootstrap sample with None as the default
    
    Returns:
        Returns a tuple containing a dataframe containing the Bootstrap X-values and a list of Bootstrap y-values
    """
    
    # by default return a bootstrap sample of the same size as the original dataset
    if not size: size = len(dataset_X)
    
    # if the X and y datasets aren't the same size, raise an exception
    if len(dataset_X) != len(dataset_y):
        raise Exception("Data size must match between dataset_X and dataset_y")
    
    # Store an ndarray of random indices
    inds_to_sample = np.random.choice(dataset_X.shape[0], size, replace = True)
    
    # Store the observations located at these indices in the Dataframe as the bootstrap sample data
    bootstrap_dataset_X = dataset_X.iloc[inds_to_sample]
    bootstrap_dataset_y = dataset_y.iloc[inds_to_sample]
    
    # return as a tuple the bootstrap samples of dataset_X as a pandas dataframe
    # and the bootstrap samples of dataset y as a numpy column vector
    return (bootstrap_dataset_X, bootstrap_dataset_y)
    
def calculate_coefficients(dataset_X, dataset_y, model):
    """Returns the Beta-coefficients of the given data fitted to the given model
    
    Args:
        dataset_X: the Pandas dataframe of feature variables and values
        dataset_y: the column vector of response values
        model: the model object to be fitted with the given data
    
    Returns:
        coefficients_dictionary: a dictionary containing the feature names as keys and the
        coefficients as values."""
    
    # Fit the data to the given model parameter
    model.fit(dataset_X, dataset_y)
    
    # Initalize dictionary of coefficents
    coefficients_dictionary = {}
    
    # Iterate through dataset coefficients and store them in the dictionary
    for index, val in enumerate(model.coef_):
        #print(val)
        coefficients_dictionary[dataset_X.columns[index]] = val
    
    # return coefficients  in the variable  coefficients_dictionary as a dictionary
    # with the key being the name of the feature as a string
    # the value being the value of the coefficients
    # do not return the intercept as part of this
    return coefficients_dictionary

def get_significant_predictors(regression_coefficients, significance_level):
    """Returns the significant predictors from a model
    
    Args:
        regression_coefficients: a list of dictionaries containing the regression coefficients
        of models made from different samples
        significance_level: the alpha-level used to determine feature significance
    
    Returns:
        significant_coefficients: a list of the significant feature names
        """
        
    # initialize a DataFrame to contain a list of the column names 
    coefficient_values = pd.DataFrame(index=list(regression_coefficients[0].keys()))
    
    # Store all of the values from each bootstrap sample in a column
    for row, cur_sample in enumerate(regression_coefficients):
        coefficient_values[row] = cur_sample.values()
        
    # Transpose the DataFrame so that each row now represents one Bootstrap sample
    coefficient_values = coefficient_values.T
    
    # Initialize significant_coefficients to an empty list
    significant_coefficients = []
    
    # Store the upper and lower percentile bounds
    lower_bound = significance_level/2 * 100
    upper_bound = 100 - lower_bound

    # Iterate through all of coefficient_values and determine which coefficients are significant
    for index_column, col in enumerate(coefficient_values): 
        if not(((np.percentile(coefficient_values[col], lower_bound ) < 0)) & ((np.percentile(coefficient_values[col], upper_bound) > 0))):
            significant_coefficients.append(coefficient_values.columns[index_column])
    
    # return significant_coefficients
    return significant_coefficients

# Initialize the list of coefficients
boot_coefs = [np.nan]*500

# Create 500 Bootstrap samples
for i in range(500):
    boot_x, boot_y = make_bootstrap_sample(X_train, y_train)
    boot_coefs[i] = calculate_coefficients(boot_x, boot_y, LinearRegression())

# Store the significant predictors
lin_missile_significant_bootstrap = get_significant_predictors(boot_coefs, 0.05)

# Print the significant predictors
print("\033[1mSignificant Predictors:\033[0m\n{0}".format(lin_missile_significant_bootstrap))

Significant Predictors:
['Unnamed: 0', 'Unnamed: 0.1', 'Bicycle_pcnt', 'Bus_pcnt', 'CarDrvr_pcnt', 'CarPsngr_pcnt', 'DistrictCode', 'FiveYrsUnknSA_pcnt', 'IndstN_pcnt', 'IndstO_pcnt', 'LaborForceY_thou', 'MetrCode', 'NatRegCode', 'OnFoot_pcnt', 'OtherCertA_pcnt', 'OtherVehicle_pcnt', 'PostSecondaryCert_pcnt', 'SubDistrictCode', 'Train_pcnt', 'TypeLocCode', 'Vehicle1up_pcnt', 'Vehicle2up_pcnt', 'WorkPlace_pcnt', 'Wrk2008Y_pcnt', 'Wrk2008_15_17_pcnt', 'age10_14_pcnt', 'age15_24_thou', 'age25_29_pcnt', 'age30_34_pcnt', 'age40_44_pcnt', 'age55_64_thou', 'age65_thou', 'age75_79_pcnt', 'age80_84_pcnt', 'age85_pcnt', 'age_median', 'aliya1947_pcnt', 'aliya1960_pcnt', 'aliya2002_pcnt', 'aliya48_54_pcnt', 'aliya55_60_pcnt', 'aliya61_71_pcnt', 'aliya72_79_pcnt', 'aliya90_01_pcnt', 'cellphone_avg', 'divorced15_24_pcnt', 'divorced35_44_pcnt', 'divorced45_54_pcnt', 'divorced55_64_pcnt', 'dress65_pcnt', 'educ13_15_pcnt', 'educ13m_pcnt', 'educ8_pcnt', 'hh0_17_4_pcnt', 'hh0_17_5_pcnt', 'hh65_pcnt', 'in

In [125]:
len(['Unnamed: 0', 'Unnamed: 0.1', 'Bicycle_pcnt', 'Bus_pcnt', 'CarDrvr_pcnt', 'CarPsngr_pcnt', 'DistrictCode', 'FiveYrsUnknSA_pcnt', 'IndstN_pcnt', 'IndstO_pcnt', 'LaborForceY_thou', 'MetrCode', 'NatRegCode', 'OnFoot_pcnt', 'OtherCertA_pcnt', 'OtherVehicle_pcnt', 'PostSecondaryCert_pcnt', 'SubDistrictCode', 'Train_pcnt', 'TypeLocCode', 'Vehicle1up_pcnt', 'Vehicle2up_pcnt', 'WorkPlace_pcnt', 'Wrk2008Y_pcnt', 'Wrk2008_15_17_pcnt', 'age10_14_pcnt', 'age15_24_thou', 'age25_29_pcnt', 'age30_34_pcnt', 'age40_44_pcnt', 'age55_64_thou', 'age65_thou', 'age75_79_pcnt', 'age80_84_pcnt', 'age85_pcnt', 'age_median', 'aliya1947_pcnt', 'aliya1960_pcnt', 'aliya2002_pcnt', 'aliya48_54_pcnt', 'aliya55_60_pcnt', 'aliya61_71_pcnt', 'aliya72_79_pcnt', 'aliya90_01_pcnt', 'cellphone_avg', 'divorced15_24_pcnt', 'divorced35_44_pcnt', 'divorced45_54_pcnt', 'divorced55_64_pcnt', 'dress65_pcnt', 'educ13_15_pcnt', 'educ13m_pcnt', 'educ8_pcnt', 'hh0_17_4_pcnt', 'hh0_17_5_pcnt', 'hh65_pcnt', 'inst_pcnt', 'm_IndstA_pcnt', 'm_IndstG_pcnt', 'm_IndstJ_pcnt', 'm_IndstO_pcnt', 'm_LaborForceY_thou', 'm_MarriageAge_mdn', 'm_Occup5_pcnt', 'm_OtherCertA_pcnt', 'm_PostSecondaryCert_pcnt', 'm_age0_4_pcnt', 'm_age10_14_pcnt', 'm_age15_24_thou', 'm_age15sef_thou', 'm_age20_24_pcnt', 'm_age30_34_pcnt', 'm_age40_44_pcnt', 'm_age55_59_pcnt', 'm_age55_64_thou', 'm_age65_69_pcnt', 'm_age65m_pcnt', 'm_age70_74_pcnt', 'm_age80_84_pcnt', 'm_divorced25_34_pcnt', 'm_divorced35_44_pcnt', 'm_educ13_15_pcnt', 'm_educ16_pcnt', 'm_educ9_12_pcnt', 'm_married15_24_pcnt', 'm_married25_34_pcnt', 'm_married35_44_pcnt', 'm_nevermarried15_24_pcnt', 'm_nevermarried25_34_pcnt', 'm_nevermarried35_44_pcnt', 'm_nevermarried45_54_pcnt', 'm_remember5_pcnt', 'm_widowed25_34_pcnt', 'married15_24_pcnt', 'married45_54_pcnt', 'married55_64_pcnt', 'nevermarried15_24_pcnt', 'nevermarried55_64_pcnt', 'remember65_pcnt', 'rm4_pcnt', 'rm5up_pcnt', 'rm6_pcnt', 'see65_pcnt', 'size5_pcnt', 'w_HrsWrkWk_avg', 'w_IndstC_pcnt', 'w_IndstF_pcnt', 'w_IndstJ_pcnt', 'w_IndstN_pcnt', 'w_LaborForceN_thou', 'w_MarriageAge_mdn', 'w_Occup2_pcnt', 'w_Occup5_pcnt', 'w_OccupX_pcnt', 'w_OtherCertA_pcnt', 'w_PostSecondaryCert_pcnt', 'w_TrnsprtMore_pcnt', 'w_TrnsprtOne_pcnt', 'w_age15_19_pcnt', 'w_age15_24_thou', 'w_age25_29_pcnt', 'w_age40_44_pcnt', 'w_age45_54_thou', 'w_age5_9_pcnt', 'w_age60_64_pcnt', 'w_age65_69_pcnt', 'w_age65_thou', 'w_age75_79_pcnt', 'w_divorced15_24_pcnt', 'w_divorced35_44_pcnt', 'w_divorced45_54_pcnt', 'w_divorced55_64_pcnt', 'w_divorced65_pcnt', 'w_divorced_pcnt', 'w_educ13_15_pcnt', 'w_educ16_pcnt', 'w_educ8_pcnt', 'w_educ9_12_pcnt', 'w_married35_44_pcnt', 'w_married45_54_pcnt', 'w_married55_64_pcnt', 'w_married65_pcnt', 'w_nevermarried15_24_pcnt', 'w_nevermarried45_54_pcnt', 'w_nevermarried55_64_pcnt', 'w_nevermarried65_pcnt', 'w_nevermarried_pcnt', 'w_notmarried_pcnt', 'w_see5_pcnt', 'w_widowed15_24_pcnt', 'w_widowed25_34_pcnt', 'w_widowed35_44_pcnt', 'w_widowed45_54_pcnt', 'w_widowed55_64_pcnt', 'w_widowed65_pcnt', 'walk5_pcnt', 'widowed15_24_pcnt', 'widowed25_34_pcnt', 'widowed45_54_pcnt', 'widowed55_64_pcnt', 'wrk2008m_pcnt', 'distance_from_gaza'])

162

In [127]:
X_sig1 = X[['Unnamed: 0', 'Unnamed: 0.1', 'Bicycle_pcnt', 'Bus_pcnt', 'CarDrvr_pcnt', 'CarPsngr_pcnt', 'DistrictCode', 'FiveYrsUnknSA_pcnt', 'IndstN_pcnt', 'IndstO_pcnt', 'LaborForceY_thou', 'MetrCode', 'NatRegCode', 'OnFoot_pcnt', 'OtherCertA_pcnt', 'OtherVehicle_pcnt', 'PostSecondaryCert_pcnt', 'SubDistrictCode', 'Train_pcnt', 'TypeLocCode', 'Vehicle1up_pcnt', 'Vehicle2up_pcnt', 'WorkPlace_pcnt', 'Wrk2008Y_pcnt', 'Wrk2008_15_17_pcnt', 'age10_14_pcnt', 'age15_24_thou', 'age25_29_pcnt', 'age30_34_pcnt', 'age40_44_pcnt', 'age55_64_thou', 'age65_thou', 'age75_79_pcnt', 'age80_84_pcnt', 'age85_pcnt', 'age_median', 'aliya1947_pcnt', 'aliya1960_pcnt', 'aliya2002_pcnt', 'aliya48_54_pcnt', 'aliya55_60_pcnt', 'aliya61_71_pcnt', 'aliya72_79_pcnt', 'aliya90_01_pcnt', 'cellphone_avg', 'divorced15_24_pcnt', 'divorced35_44_pcnt', 'divorced45_54_pcnt', 'divorced55_64_pcnt', 'dress65_pcnt', 'educ13_15_pcnt', 'educ13m_pcnt', 'educ8_pcnt', 'hh0_17_4_pcnt', 'hh0_17_5_pcnt', 'hh65_pcnt', 'inst_pcnt', 'm_IndstA_pcnt', 'm_IndstG_pcnt', 'm_IndstJ_pcnt', 'm_IndstO_pcnt', 'm_LaborForceY_thou', 'm_MarriageAge_mdn', 'm_Occup5_pcnt', 'm_OtherCertA_pcnt', 'm_PostSecondaryCert_pcnt', 'm_age0_4_pcnt', 'm_age10_14_pcnt', 'm_age15_24_thou', 'm_age15sef_thou', 'm_age20_24_pcnt', 'm_age30_34_pcnt', 'm_age40_44_pcnt', 'm_age55_59_pcnt', 'm_age55_64_thou', 'm_age65_69_pcnt', 'm_age65m_pcnt', 'm_age70_74_pcnt', 'm_age80_84_pcnt', 'm_divorced25_34_pcnt', 'm_divorced35_44_pcnt', 'm_educ13_15_pcnt', 'm_educ16_pcnt', 'm_educ9_12_pcnt', 'm_married15_24_pcnt', 'm_married25_34_pcnt', 'm_married35_44_pcnt', 'm_nevermarried15_24_pcnt', 'm_nevermarried25_34_pcnt', 'm_nevermarried35_44_pcnt', 'm_nevermarried45_54_pcnt', 'm_remember5_pcnt', 'm_widowed25_34_pcnt', 'married15_24_pcnt', 'married45_54_pcnt', 'married55_64_pcnt', 'nevermarried15_24_pcnt', 'nevermarried55_64_pcnt', 'remember65_pcnt', 'rm4_pcnt', 'rm5up_pcnt', 'rm6_pcnt', 'see65_pcnt', 'size5_pcnt', 'w_HrsWrkWk_avg', 'w_IndstC_pcnt', 'w_IndstF_pcnt', 'w_IndstJ_pcnt', 'w_IndstN_pcnt', 'w_LaborForceN_thou', 'w_MarriageAge_mdn', 'w_Occup2_pcnt', 'w_Occup5_pcnt', 'w_OccupX_pcnt', 'w_OtherCertA_pcnt', 'w_PostSecondaryCert_pcnt', 'w_TrnsprtMore_pcnt', 'w_TrnsprtOne_pcnt', 'w_age15_19_pcnt', 'w_age15_24_thou', 'w_age25_29_pcnt', 'w_age40_44_pcnt', 'w_age45_54_thou', 'w_age5_9_pcnt', 'w_age60_64_pcnt', 'w_age65_69_pcnt', 'w_age65_thou', 'w_age75_79_pcnt', 'w_divorced15_24_pcnt', 'w_divorced35_44_pcnt', 'w_divorced45_54_pcnt', 'w_divorced55_64_pcnt', 'w_divorced65_pcnt', 'w_divorced_pcnt', 'w_educ13_15_pcnt', 'w_educ16_pcnt', 'w_educ8_pcnt', 'w_educ9_12_pcnt', 'w_married35_44_pcnt', 'w_married45_54_pcnt', 'w_married55_64_pcnt', 'w_married65_pcnt', 'w_nevermarried15_24_pcnt', 'w_nevermarried45_54_pcnt', 'w_nevermarried55_64_pcnt', 'w_nevermarried65_pcnt', 'w_nevermarried_pcnt', 'w_notmarried_pcnt', 'w_see5_pcnt', 'w_widowed15_24_pcnt', 'w_widowed25_34_pcnt', 'w_widowed35_44_pcnt', 'w_widowed45_54_pcnt', 'w_widowed55_64_pcnt', 'w_widowed65_pcnt', 'walk5_pcnt', 'widowed15_24_pcnt', 'widowed25_34_pcnt', 'widowed45_54_pcnt', 'widowed55_64_pcnt', 'wrk2008m_pcnt', 'distance_from_gaza']]

In [128]:
# re split
X_train, X_test, y_train, y_test = train_test_split(X_sig1, y, test_size=0.2, random_state=42)

# Initialize the list of coefficients
boot_coefs = [np.nan]*500

# Create 500 Bootstrap samples
for i in range(500):
    boot_x, boot_y = make_bootstrap_sample(X_train, y_train)
    boot_coefs[i] = calculate_coefficients(boot_x, boot_y, LinearRegression())

# Store the significant predictors
lin_missile_significant_bootstrap = get_significant_predictors(boot_coefs, 0.05)


# Print the significant predictors
print("\033[1mSignificant Predictors:\033[0m\n{0}".format(lin_missile_significant_bootstrap))

Significant Predictors:
['Unnamed: 0', 'Unnamed: 0.1', 'Bicycle_pcnt', 'Bus_pcnt', 'CarDrvr_pcnt', 'CarPsngr_pcnt', 'DistrictCode', 'IndstO_pcnt', 'LaborForceY_thou', 'MetrCode', 'NatRegCode', 'OnFoot_pcnt', 'OtherCertA_pcnt', 'OtherVehicle_pcnt', 'PostSecondaryCert_pcnt', 'SubDistrictCode', 'Train_pcnt', 'TypeLocCode', 'Vehicle1up_pcnt', 'Vehicle2up_pcnt', 'WorkPlace_pcnt', 'Wrk2008Y_pcnt', 'Wrk2008_15_17_pcnt', 'age10_14_pcnt', 'age15_24_thou', 'age25_29_pcnt', 'age30_34_pcnt', 'age40_44_pcnt', 'age55_64_thou', 'age65_thou', 'age75_79_pcnt', 'age80_84_pcnt', 'age_median', 'aliya1947_pcnt', 'aliya55_60_pcnt', 'aliya61_71_pcnt', 'cellphone_avg', 'divorced35_44_pcnt', 'divorced45_54_pcnt', 'divorced55_64_pcnt', 'dress65_pcnt', 'educ13_15_pcnt', 'educ13m_pcnt', 'educ8_pcnt', 'hh0_17_5_pcnt', 'hh65_pcnt', 'm_IndstG_pcnt', 'm_IndstJ_pcnt', 'm_IndstO_pcnt', 'm_LaborForceY_thou', 'm_MarriageAge_mdn', 'm_Occup5_pcnt', 'm_OtherCertA_pcnt', 'm_PostSecondaryCert_pcnt', 'm_age0_4_pcnt', 'm_age10_

In [129]:
len(['Unnamed: 0', 'Unnamed: 0.1', 'Bicycle_pcnt', 'Bus_pcnt', 'CarDrvr_pcnt', 'CarPsngr_pcnt', 'DistrictCode', 'IndstO_pcnt', 'LaborForceY_thou', 'MetrCode', 'NatRegCode', 'OnFoot_pcnt', 'OtherCertA_pcnt', 'OtherVehicle_pcnt', 'PostSecondaryCert_pcnt', 'SubDistrictCode', 'Train_pcnt', 'TypeLocCode', 'Vehicle1up_pcnt', 'Vehicle2up_pcnt', 'WorkPlace_pcnt', 'Wrk2008Y_pcnt', 'Wrk2008_15_17_pcnt', 'age10_14_pcnt', 'age15_24_thou', 'age25_29_pcnt', 'age30_34_pcnt', 'age40_44_pcnt', 'age55_64_thou', 'age65_thou', 'age75_79_pcnt', 'age80_84_pcnt', 'age_median', 'aliya1947_pcnt', 'aliya55_60_pcnt', 'aliya61_71_pcnt', 'cellphone_avg', 'divorced35_44_pcnt', 'divorced45_54_pcnt', 'divorced55_64_pcnt', 'dress65_pcnt', 'educ13_15_pcnt', 'educ13m_pcnt', 'educ8_pcnt', 'hh0_17_5_pcnt', 'hh65_pcnt', 'm_IndstG_pcnt', 'm_IndstJ_pcnt', 'm_IndstO_pcnt', 'm_LaborForceY_thou', 'm_MarriageAge_mdn', 'm_Occup5_pcnt', 'm_OtherCertA_pcnt', 'm_PostSecondaryCert_pcnt', 'm_age0_4_pcnt', 'm_age10_14_pcnt', 'm_age15_24_thou', 'm_age15sef_thou', 'm_age30_34_pcnt', 'm_age40_44_pcnt', 'm_age55_59_pcnt', 'm_age55_64_thou', 'm_age65_69_pcnt', 'm_age65m_pcnt', 'm_age70_74_pcnt', 'm_divorced25_34_pcnt', 'm_divorced35_44_pcnt', 'm_educ13_15_pcnt', 'm_educ16_pcnt', 'm_educ9_12_pcnt', 'm_married25_34_pcnt', 'm_married35_44_pcnt', 'm_nevermarried15_24_pcnt', 'm_nevermarried25_34_pcnt', 'm_nevermarried45_54_pcnt', 'm_remember5_pcnt', 'm_widowed25_34_pcnt', 'married15_24_pcnt', 'married45_54_pcnt', 'married55_64_pcnt', 'nevermarried15_24_pcnt', 'nevermarried55_64_pcnt', 'remember65_pcnt', 'rm4_pcnt', 'rm6_pcnt', 'see65_pcnt', 'size5_pcnt', 'w_HrsWrkWk_avg', 'w_IndstC_pcnt', 'w_IndstF_pcnt', 'w_LaborForceN_thou', 'w_Occup5_pcnt', 'w_OccupX_pcnt', 'w_OtherCertA_pcnt', 'w_PostSecondaryCert_pcnt', 'w_age15_24_thou', 'w_age40_44_pcnt', 'w_age5_9_pcnt', 'w_age60_64_pcnt', 'w_age65_69_pcnt', 'w_age65_thou', 'w_divorced35_44_pcnt', 'w_divorced55_64_pcnt', 'w_divorced65_pcnt', 'w_educ13_15_pcnt', 'w_educ16_pcnt', 'w_educ8_pcnt', 'w_educ9_12_pcnt', 'w_married55_64_pcnt', 'w_married65_pcnt', 'w_nevermarried15_24_pcnt', 'w_nevermarried65_pcnt', 'w_nevermarried_pcnt', 'w_see5_pcnt', 'w_widowed55_64_pcnt', 'w_widowed65_pcnt', 'walk5_pcnt', 'widowed25_34_pcnt', 'widowed45_54_pcnt', 'widowed55_64_pcnt', 'wrk2008m_pcnt', 'distance_from_gaza'])

122

In [133]:
X_sig2 = X[['Bicycle_pcnt', 'Bus_pcnt', 'CarDrvr_pcnt', 'CarPsngr_pcnt', 'DistrictCode', 'IndstO_pcnt', 'LaborForceY_thou', 'MetrCode', 'NatRegCode', 'OnFoot_pcnt', 'OtherCertA_pcnt', 'OtherVehicle_pcnt', 'PostSecondaryCert_pcnt', 'SubDistrictCode', 'Train_pcnt', 'TypeLocCode', 'Vehicle1up_pcnt', 'Vehicle2up_pcnt', 'WorkPlace_pcnt', 'Wrk2008Y_pcnt', 'Wrk2008_15_17_pcnt', 'age10_14_pcnt', 'age15_24_thou', 'age25_29_pcnt', 'age30_34_pcnt', 'age40_44_pcnt', 'age55_64_thou', 'age65_thou', 'age75_79_pcnt', 'age80_84_pcnt', 'age_median', 'aliya1947_pcnt', 'aliya55_60_pcnt', 'aliya61_71_pcnt', 'cellphone_avg', 'divorced35_44_pcnt', 'divorced45_54_pcnt', 'divorced55_64_pcnt', 'dress65_pcnt', 'educ13_15_pcnt', 'educ13m_pcnt', 'educ8_pcnt', 'hh0_17_5_pcnt', 'hh65_pcnt', 'm_IndstG_pcnt', 'm_IndstJ_pcnt', 'm_IndstO_pcnt', 'm_LaborForceY_thou', 'm_MarriageAge_mdn', 'm_Occup5_pcnt', 'm_OtherCertA_pcnt', 'm_PostSecondaryCert_pcnt', 'm_age0_4_pcnt', 'm_age10_14_pcnt', 'm_age15_24_thou', 'm_age15sef_thou', 'm_age30_34_pcnt', 'm_age40_44_pcnt', 'm_age55_59_pcnt', 'm_age55_64_thou', 'm_age65_69_pcnt', 'm_age65m_pcnt', 'm_age70_74_pcnt', 'm_divorced25_34_pcnt', 'm_divorced35_44_pcnt', 'm_educ13_15_pcnt', 'm_educ16_pcnt', 'm_educ9_12_pcnt', 'm_married25_34_pcnt', 'm_married35_44_pcnt', 'm_nevermarried15_24_pcnt', 'm_nevermarried25_34_pcnt', 'm_nevermarried45_54_pcnt', 'm_remember5_pcnt', 'm_widowed25_34_pcnt', 'married15_24_pcnt', 'married45_54_pcnt', 'married55_64_pcnt', 'nevermarried15_24_pcnt', 'nevermarried55_64_pcnt', 'remember65_pcnt', 'rm4_pcnt', 'rm6_pcnt', 'see65_pcnt', 'size5_pcnt', 'w_HrsWrkWk_avg', 'w_IndstC_pcnt', 'w_IndstF_pcnt', 'w_LaborForceN_thou', 'w_Occup5_pcnt', 'w_OccupX_pcnt', 'w_OtherCertA_pcnt', 'w_PostSecondaryCert_pcnt', 'w_age15_24_thou', 'w_age40_44_pcnt', 'w_age5_9_pcnt', 'w_age60_64_pcnt', 'w_age65_69_pcnt', 'w_age65_thou', 'w_divorced35_44_pcnt', 'w_divorced55_64_pcnt', 'w_divorced65_pcnt', 'w_educ13_15_pcnt', 'w_educ16_pcnt', 'w_educ8_pcnt', 'w_educ9_12_pcnt', 'w_married55_64_pcnt', 'w_married65_pcnt', 'w_nevermarried15_24_pcnt', 'w_nevermarried65_pcnt', 'w_nevermarried_pcnt', 'w_see5_pcnt', 'w_widowed55_64_pcnt', 'w_widowed65_pcnt', 'walk5_pcnt', 'widowed25_34_pcnt', 'widowed45_54_pcnt', 'widowed55_64_pcnt', 'wrk2008m_pcnt', 'distance_from_gaza']]

In [134]:
# re split
X_train, X_test, y_train, y_test = train_test_split(X_sig2, y, test_size=0.2, random_state=42)

# Initialize the list of coefficients
boot_coefs = [np.nan]*500

# Create 500 Bootstrap samples
for i in range(500):
    boot_x, boot_y = make_bootstrap_sample(X_train, y_train)
    boot_coefs[i] = calculate_coefficients(boot_x, boot_y, LinearRegression())

# Store the significant predictors
lin_missile_significant_bootstrap = get_significant_predictors(boot_coefs, 0.05)


# Print the significant predictors
print("\033[1mSignificant Predictors:\033[0m\n{0}".format(lin_missile_significant_bootstrap))

Significant Predictors:
['Bicycle_pcnt', 'Bus_pcnt', 'CarDrvr_pcnt', 'CarPsngr_pcnt', 'DistrictCode', 'IndstO_pcnt', 'LaborForceY_thou', 'MetrCode', 'NatRegCode', 'OnFoot_pcnt', 'OtherCertA_pcnt', 'OtherVehicle_pcnt', 'PostSecondaryCert_pcnt', 'SubDistrictCode', 'Train_pcnt', 'TypeLocCode', 'Vehicle1up_pcnt', 'Vehicle2up_pcnt', 'WorkPlace_pcnt', 'Wrk2008Y_pcnt', 'age10_14_pcnt', 'age15_24_thou', 'age25_29_pcnt', 'age30_34_pcnt', 'age40_44_pcnt', 'age55_64_thou', 'age65_thou', 'age75_79_pcnt', 'age_median', 'aliya1947_pcnt', 'aliya55_60_pcnt', 'aliya61_71_pcnt', 'cellphone_avg', 'divorced35_44_pcnt', 'divorced45_54_pcnt', 'divorced55_64_pcnt', 'dress65_pcnt', 'educ8_pcnt', 'hh0_17_5_pcnt', 'hh65_pcnt', 'm_IndstG_pcnt', 'm_IndstJ_pcnt', 'm_IndstO_pcnt', 'm_LaborForceY_thou', 'm_MarriageAge_mdn', 'm_Occup5_pcnt', 'm_OtherCertA_pcnt', 'm_PostSecondaryCert_pcnt', 'm_age0_4_pcnt', 'm_age10_14_pcnt', 'm_age15_24_thou', 'm_age15sef_thou', 'm_age30_34_pcnt', 'm_age40_44_pcnt', 'm_age55_59_pcnt'

In [136]:
sig_filter = ['Bicycle_pcnt', 'Bus_pcnt', 'CarDrvr_pcnt', 'CarPsngr_pcnt', 'DistrictCode', 'IndstO_pcnt', 'LaborForceY_thou', 'MetrCode', 'NatRegCode', 'OnFoot_pcnt', 'OtherCertA_pcnt', 'OtherVehicle_pcnt', 'PostSecondaryCert_pcnt', 'SubDistrictCode', 'Train_pcnt', 'TypeLocCode', 'Vehicle1up_pcnt', 'Vehicle2up_pcnt', 'WorkPlace_pcnt', 'Wrk2008Y_pcnt', 'age10_14_pcnt', 'age15_24_thou', 'age25_29_pcnt', 'age30_34_pcnt', 'age40_44_pcnt', 'age55_64_thou', 'age65_thou', 'age75_79_pcnt', 'age_median', 'aliya1947_pcnt', 'aliya55_60_pcnt', 'aliya61_71_pcnt', 'cellphone_avg', 'divorced35_44_pcnt', 'divorced45_54_pcnt', 'divorced55_64_pcnt', 'dress65_pcnt', 'educ8_pcnt', 'hh0_17_5_pcnt', 'hh65_pcnt', 'm_IndstG_pcnt', 'm_IndstJ_pcnt', 'm_IndstO_pcnt', 'm_LaborForceY_thou', 'm_MarriageAge_mdn', 'm_Occup5_pcnt', 'm_OtherCertA_pcnt', 'm_PostSecondaryCert_pcnt', 'm_age0_4_pcnt', 'm_age10_14_pcnt', 'm_age15_24_thou', 'm_age15sef_thou', 'm_age30_34_pcnt', 'm_age40_44_pcnt', 'm_age55_59_pcnt', 'm_age55_64_thou', 'm_age65_69_pcnt', 'm_age65m_pcnt', 'm_divorced25_34_pcnt', 'm_divorced35_44_pcnt', 'm_educ13_15_pcnt', 'm_educ16_pcnt', 'm_educ9_12_pcnt', 'm_married25_34_pcnt', 'm_married35_44_pcnt', 'm_nevermarried15_24_pcnt', 'm_nevermarried25_34_pcnt', 'm_nevermarried45_54_pcnt', 'm_remember5_pcnt', 'married15_24_pcnt', 'married45_54_pcnt', 'married55_64_pcnt', 'nevermarried15_24_pcnt', 'nevermarried55_64_pcnt', 'rm4_pcnt', 'rm6_pcnt', 'see65_pcnt', 'size5_pcnt', 'w_HrsWrkWk_avg', 'w_IndstF_pcnt', 'w_LaborForceN_thou', 'w_OccupX_pcnt', 'w_OtherCertA_pcnt', 'w_PostSecondaryCert_pcnt', 'w_age15_24_thou', 'w_age40_44_pcnt', 'w_age5_9_pcnt', 'w_age60_64_pcnt', 'w_age65_69_pcnt', 'w_age65_thou', 'w_divorced35_44_pcnt', 'w_divorced55_64_pcnt', 'w_educ13_15_pcnt', 'w_educ8_pcnt', 'w_married55_64_pcnt', 'w_nevermarried15_24_pcnt', 'w_nevermarried65_pcnt', 'w_nevermarried_pcnt', 'w_see5_pcnt', 'w_widowed55_64_pcnt', 'walk5_pcnt', 'widowed25_34_pcnt', 'widowed45_54_pcnt', 'widowed55_64_pcnt', 'wrk2008m_pcnt']

X_sig3 = X[sig_filter]

In [140]:
scaler = MinMaxScaler()
X_sig3_normalized = pd.DataFrame(data = scaler.fit_transform(X_sig3), columns = sig_filter)

In [141]:
# re split
X_train, X_test, y_train, y_test = train_test_split(X_sig3_normalized, y, test_size=0.2, random_state=42)

# Initialize the list of coefficients
boot_coefs = [np.nan]*500

# Create 500 Bootstrap samples
for i in range(500):
    boot_x, boot_y = make_bootstrap_sample(X_train, y_train)
    boot_coefs[i] = calculate_coefficients(boot_x, boot_y, LinearRegression())

# Store the significant predictors
lin_missile_significant_bootstrap = get_significant_predictors(boot_coefs, 0.05)


# Print the significant predictors
print("\033[1mSignificant Predictors:\033[0m\n{0}".format(lin_missile_significant_bootstrap))

Significant Predictors:
['Bicycle_pcnt', 'Bus_pcnt', 'CarDrvr_pcnt', 'CarPsngr_pcnt', 'DistrictCode', 'IndstO_pcnt', 'LaborForceY_thou', 'MetrCode', 'NatRegCode', 'OnFoot_pcnt', 'OtherCertA_pcnt', 'OtherVehicle_pcnt', 'PostSecondaryCert_pcnt', 'SubDistrictCode', 'Train_pcnt', 'TypeLocCode', 'Vehicle1up_pcnt', 'Vehicle2up_pcnt', 'WorkPlace_pcnt', 'Wrk2008Y_pcnt', 'age10_14_pcnt', 'age15_24_thou', 'age25_29_pcnt', 'age30_34_pcnt', 'age40_44_pcnt', 'age55_64_thou', 'age65_thou', 'age75_79_pcnt', 'age_median', 'aliya1947_pcnt', 'aliya55_60_pcnt', 'aliya61_71_pcnt', 'cellphone_avg', 'divorced35_44_pcnt', 'divorced45_54_pcnt', 'divorced55_64_pcnt', 'educ8_pcnt', 'hh0_17_5_pcnt', 'hh65_pcnt', 'm_IndstG_pcnt', 'm_IndstJ_pcnt', 'm_IndstO_pcnt', 'm_LaborForceY_thou', 'm_MarriageAge_mdn', 'm_Occup5_pcnt', 'm_OtherCertA_pcnt', 'm_PostSecondaryCert_pcnt', 'm_age0_4_pcnt', 'm_age10_14_pcnt', 'm_age15_24_thou', 'm_age15sef_thou', 'm_age30_34_pcnt', 'm_age40_44_pcnt', 'm_age55_59_pcnt', 'm_age55_64_th

In [143]:
features_final = X_sig3_normalized[['Bicycle_pcnt', 'Bus_pcnt', 'CarDrvr_pcnt', 'CarPsngr_pcnt', 'DistrictCode', 'IndstO_pcnt', 'LaborForceY_thou', 'MetrCode', 'NatRegCode', 'OnFoot_pcnt', 'OtherCertA_pcnt', 'OtherVehicle_pcnt', 'PostSecondaryCert_pcnt', 'SubDistrictCode', 'Train_pcnt', 'TypeLocCode', 'Vehicle1up_pcnt', 'Vehicle2up_pcnt', 'WorkPlace_pcnt', 'Wrk2008Y_pcnt', 'age10_14_pcnt', 'age15_24_thou', 'age25_29_pcnt', 'age30_34_pcnt', 'age40_44_pcnt', 'age55_64_thou', 'age65_thou', 'age75_79_pcnt', 'age_median', 'aliya1947_pcnt', 'aliya55_60_pcnt', 'aliya61_71_pcnt', 'cellphone_avg', 'divorced35_44_pcnt', 'divorced45_54_pcnt', 'divorced55_64_pcnt', 'educ8_pcnt', 'hh0_17_5_pcnt', 'hh65_pcnt', 'm_IndstG_pcnt', 'm_IndstJ_pcnt', 'm_IndstO_pcnt', 'm_LaborForceY_thou', 'm_MarriageAge_mdn', 'm_Occup5_pcnt', 'm_OtherCertA_pcnt', 'm_PostSecondaryCert_pcnt', 'm_age0_4_pcnt', 'm_age10_14_pcnt', 'm_age15_24_thou', 'm_age15sef_thou', 'm_age30_34_pcnt', 'm_age40_44_pcnt', 'm_age55_59_pcnt', 'm_age55_64_thou', 'm_age65_69_pcnt', 'm_age65m_pcnt', 'm_divorced35_44_pcnt', 'm_educ13_15_pcnt', 'm_educ16_pcnt', 'm_educ9_12_pcnt', 'm_married25_34_pcnt', 'm_married35_44_pcnt', 'm_nevermarried15_24_pcnt', 'm_nevermarried25_34_pcnt', 'm_nevermarried45_54_pcnt', 'm_remember5_pcnt', 'married15_24_pcnt', 'married45_54_pcnt', 'married55_64_pcnt', 'nevermarried15_24_pcnt', 'nevermarried55_64_pcnt', 'rm4_pcnt', 'rm6_pcnt', 'w_HrsWrkWk_avg', 'w_IndstF_pcnt', 'w_LaborForceN_thou', 'w_OccupX_pcnt', 'w_OtherCertA_pcnt', 'w_PostSecondaryCert_pcnt', 'w_age15_24_thou', 'w_age40_44_pcnt', 'w_age5_9_pcnt', 'w_age60_64_pcnt', 'w_age65_69_pcnt', 'w_age65_thou', 'w_divorced35_44_pcnt', 'w_divorced55_64_pcnt', 'w_educ8_pcnt', 'w_married55_64_pcnt', 'w_nevermarried15_24_pcnt', 'w_nevermarried65_pcnt', 'w_nevermarried_pcnt', 'w_see5_pcnt', 'w_widowed55_64_pcnt', 'walk5_pcnt', 'widowed25_34_pcnt', 'widowed45_54_pcnt', 'widowed55_64_pcnt', 'wrk2008m_pcnt']]

In [144]:
# re split
X_train, X_test, y_train, y_test = train_test_split(features_final, y, test_size=0.2, random_state=42)

In [150]:
final_model = LinearRegression()
final_model.fit(X=X_train, y=y_train)

print("\033[1mR^2 Score on Test Set for Linear Model Predicting Rocket Instances:\033[0m {0}".format(r2_score(y_test, final_model.predict(X_test))))

R^2 Score on Test Set for Linear Model Predicting Rocket Instances: 0.8882407721013853


In [151]:
print(final_model.predict(X_test))

[ 88.73004869 277.31495102 -16.0637925  ...  74.86567814 110.16218165
 171.4362997 ]


In [152]:
cleaned_data = features_final.copy()

In [153]:
features_final["count"] = y

In [156]:
features_final.to_csv("FINAL_DATA_USED_FOR_LINEAR_REGRESSION.csv")